In [ ]:
from component import io 
from component import tile

In [ ]:
export_io = io.ExportIo()

In [ ]:
# for debug
#import pandas as pd 
#import geopandas as gpd
#from component import parameter as cp
#
#tb_io = io.TableIo()
#tb_io.json_table = '{"pathname": "/home/prambaud/modules/clip-time-series/utils/clip_test_points.csv", "id_column": "id", "lat_column": "Y", "lng_column": "X"}'
#
#df = pd.read_csv(cp.test_dataset, sep=None, engine='python')
#df = df.filter(items=["Y", "X", "id"])
#df = df.rename(columns={"Y": 'lat', "X": 'lng', "id": 'id'})
#gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lng, df.lat), crs='EPSG:4326')
#tb_io.pts = gdf 
#
#viz_io = io.VizIo()
#viz_io.check = True
#viz_io.sources = ['landsat']
#
#viz_io.bands = 'Red, Green, Blue'
#viz_io.start_year =  2013
#viz_io.end_year = 2019
#viz_io.square_size = 2000

In [ ]:
#result tile 
export_results = tile.ExportResult()

In [ ]:
#export data
export_tile = tile.ExportData(export_io, viz_io, tb_io, export_results)

In [ ]:
export_tile

In [ ]:
export_results